# Day 4: FinBERT and Transformer Models for Financial Text

## Week 19: NLP & Alternative Data

---

### Learning Objectives
- Understand transformer architecture and self-attention mechanism
- Master FinBERT for financial sentiment analysis
- Learn tokenization strategies for financial text
- Build sentiment classification pipelines for news and earnings calls
- Fine-tune transformers on custom financial datasets
- Extract embeddings for trading signals and document similarity

### Key Interview Topics
- BERT vs traditional NLP methods (TF-IDF, Word2Vec)
- Self-attention mechanism and positional encoding
- Transfer learning in NLP for finance
- Fine-tuning strategies and domain adaptation
- Handling financial jargon and numerical data in text

---

## 1. Import Required Libraries

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR, SequentialLR, ConstantLR

# HuggingFace Transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification,
    AutoTokenizer, AutoModelForSequenceClassification,
    pipeline, Trainer, TrainingArguments,
    get_linear_schedule_with_warmup
)

# Scikit-learn utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    accuracy_score, f1_score, precision_recall_fscore_support
)
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

# Progress bars
from tqdm.auto import tqdm

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

# Plotting settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## 2. Understanding Transformer Architecture

### The Revolution in NLP

The Transformer architecture, introduced in "Attention Is All You Need" (Vaswani et al., 2017), revolutionized NLP by replacing recurrence with **self-attention**, enabling:

1. **Parallelization**: Process entire sequences simultaneously
2. **Long-range dependencies**: Attend to any position in the sequence
3. **Transfer learning**: Pre-train on massive corpora, fine-tune for specific tasks

### Key Components:
- **Self-Attention Mechanism**: Weighs importance of different tokens
- **Positional Encoding**: Injects sequence order information
- **Multi-Head Attention**: Multiple attention patterns in parallel
- **Feed-Forward Networks**: Non-linear transformations at each position

In [ ]:
class SelfAttention(nn.Module):
    """
    Simplified Self-Attention mechanism for educational purposes.
    
    Key Concepts:
    - Query (Q): "What am I looking for?"
    - Key (K): "What do I contain?"
    - Value (V): "What information do I provide?"
    
    Attention(Q, K, V) = softmax(QK^T / sqrt(d_k)) * V
    """
    
    def __init__(self, embed_dim, num_heads=1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Linear projections for Q, K, V
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
        self.W_o = nn.Linear(embed_dim, embed_dim)
        
        self.scale = self.head_dim ** -0.5
        
    def forward(self, x, return_attention=False):
        batch_size, seq_len, _ = x.shape
        
        # Project to Q, K, V
        Q = self.W_q(x)  # (batch, seq_len, embed_dim)
        K = self.W_k(x)
        V = self.W_v(x)
        
        # Compute attention scores
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scale
        attention_weights = F.softmax(attention_scores, dim=-1)
        
        # Apply attention to values
        output = torch.matmul(attention_weights, V)
        output = self.W_o(output)
        
        if return_attention:
            return output, attention_weights
        return output

# Demonstrate self-attention on a simple example
print("=" * 60)
print("SELF-ATTENTION MECHANISM DEMONSTRATION")
print("=" * 60)

# Create a simple embedding (3 tokens, 4-dimensional embeddings)
embed_dim = 4
seq_len = 3
batch_size = 1

# Simulated token embeddings for: ["stock", "price", "rises"]
sample_embeddings = torch.tensor([
    [[0.5, 0.3, 0.1, 0.8],   # "stock"
     [0.2, 0.9, 0.4, 0.1],   # "price"  
     [0.7, 0.2, 0.6, 0.3]]   # "rises"
], dtype=torch.float32)

# Apply self-attention
attention_layer = SelfAttention(embed_dim=embed_dim)
output, attention_weights = attention_layer(sample_embeddings, return_attention=True)

print(f"\nInput shape: {sample_embeddings.shape}")
print(f"Output shape: {output.shape}")
print(f"\nAttention weights (how much each token attends to others):")
print(attention_weights.squeeze().detach().numpy().round(3))

# Visualize attention
fig, ax = plt.subplots(figsize=(6, 5))
tokens = ["stock", "price", "rises"]
sns.heatmap(
    attention_weights.squeeze().detach().numpy(),
    xticklabels=tokens,
    yticklabels=tokens,
    annot=True,
    fmt='.3f',
    cmap='Blues',
    ax=ax
)
ax.set_xlabel('Key (attending to)')
ax.set_ylabel('Query (from)')
ax.set_title('Self-Attention Weights\n(Row shows what each token attends to)')
plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("Each row shows how much attention a token pays to other tokens.")
print("This allows the model to create context-aware representations.")

In [ ]:
class PositionalEncoding(nn.Module):
    """
    Positional Encoding adds position information to embeddings.
    
    Since transformers process all tokens in parallel (no recurrence),
    we need to explicitly inject position information.
    
    PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
    PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
    """
    
    def __init__(self, d_model, max_len=512):
        super().__init__()
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

# Visualize positional encodings
print("=" * 60)
print("POSITIONAL ENCODING VISUALIZATION")
print("=" * 60)

d_model = 64
max_len = 100
pe_layer = PositionalEncoding(d_model, max_len)

# Get the positional encodings
pe = pe_layer.pe.squeeze().numpy()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot first few dimensions
ax1 = axes[0]
for i in range(8):
    ax1.plot(pe[:50, i], label=f'dim {i}', alpha=0.7)
ax1.set_xlabel('Position')
ax1.set_ylabel('Encoding Value')
ax1.set_title('Positional Encoding Values (First 8 Dimensions)')
ax1.legend(loc='upper right', ncol=2, fontsize=8)
ax1.grid(True, alpha=0.3)

# Heatmap of all encodings
ax2 = axes[1]
im = ax2.imshow(pe[:50, :32].T, aspect='auto', cmap='RdBu_r')
ax2.set_xlabel('Position in Sequence')
ax2.set_ylabel('Embedding Dimension')
ax2.set_title('Positional Encoding Heatmap')
plt.colorbar(im, ax=ax2)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight:")
print("- Different dimensions have different frequencies (sine/cosine waves)")
print("- The model can learn to attend to relative positions")
print("- Positions close together have similar encodings")

## 3. Load Pre-trained BERT Model

### BERT: Bidirectional Encoder Representations from Transformers

BERT was pre-trained on two tasks:
1. **Masked Language Modeling (MLM)**: Predict masked tokens using bidirectional context
2. **Next Sentence Prediction (NSP)**: Predict if sentence B follows sentence A

Key characteristics:
- **bert-base**: 12 layers, 768 hidden, 12 attention heads, 110M parameters
- **bert-large**: 24 layers, 1024 hidden, 16 attention heads, 340M parameters

In [ ]:
# Load base BERT model and tokenizer
print("Loading BERT base model...")
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Move model to device
bert_model = bert_model.to(device)
bert_model.eval()

# Explore model architecture
print("\n" + "=" * 60)
print("BERT BASE MODEL ARCHITECTURE")
print("=" * 60)

config = bert_model.config
print(f"\nModel Configuration:")
print(f"  - Hidden size: {config.hidden_size}")
print(f"  - Number of layers: {config.num_hidden_layers}")
print(f"  - Attention heads: {config.num_attention_heads}")
print(f"  - Intermediate size: {config.intermediate_size}")
print(f"  - Vocabulary size: {config.vocab_size}")
print(f"  - Max position embeddings: {config.max_position_embeddings}")

# Count parameters
total_params = sum(p.numel() for p in bert_model.parameters())
trainable_params = sum(p.numel() for p in bert_model.parameters() if p.requires_grad)
print(f"\nParameter count:")
print(f"  - Total parameters: {total_params:,}")
print(f"  - Trainable parameters: {trainable_params:,}")

In [ ]:
# Example: Process text through BERT
sample_text = "The Federal Reserve raised interest rates by 25 basis points."

# Tokenize
inputs = bert_tokenizer(
    sample_text,
    return_tensors='pt',
    padding=True,
    truncation=True,
    max_length=512
)

print("=" * 60)
print("BERT TOKENIZATION AND OUTPUT")
print("=" * 60)
print(f"\nOriginal text: {sample_text}")
print(f"\nTokenized IDs: {inputs['input_ids'].tolist()}")
print(f"Tokens: {bert_tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])}")
print(f"Attention mask: {inputs['attention_mask'].tolist()}")

# Get BERT output
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = bert_model(**inputs)

# BERT returns:
# - last_hidden_state: (batch, seq_len, hidden_size) - contextual embeddings for each token
# - pooler_output: (batch, hidden_size) - [CLS] token transformed for classification

print(f"\nOutput shapes:")
print(f"  - Last hidden state: {outputs.last_hidden_state.shape}")
print(f"  - Pooler output: {outputs.pooler_output.shape}")

print("\n💡 Key Outputs:")
print("  - last_hidden_state: Contextual embeddings for ALL tokens")
print("  - pooler_output: [CLS] token embedding for classification tasks")

## 4. Load FinBERT for Financial Sentiment Analysis

### FinBERT: Domain-Adapted BERT for Finance

**FinBERT** (ProsusAI/finbert) is BERT fine-tuned on financial text:
- Pre-trained on financial news and communications
- Fine-tuned for sentiment classification (positive, negative, neutral)
- Better understanding of financial jargon, company names, and market terminology

### Why Domain-Specific Models Matter in Finance:
- "Bull" in finance ≠ "bull" in general English
- Numbers and percentages carry different meanings
- Subtle language differences ("missed expectations" vs "beat estimates")
- Temporal references to earnings, quarters, fiscal years

In [ ]:
# Load FinBERT model and tokenizer
print("Loading FinBERT model (ProsusAI/finbert)...")
finbert_tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')
finbert_model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')

# Move to device
finbert_model = finbert_model.to(device)
finbert_model.eval()

# Model info
print("\n" + "=" * 60)
print("FINBERT MODEL ARCHITECTURE")
print("=" * 60)

config = finbert_model.config
print(f"\nModel type: {config.model_type}")
print(f"Number of labels: {config.num_labels}")
print(f"Label mapping: {config.id2label}")

# Parameter count
total_params = sum(p.numel() for p in finbert_model.parameters())
print(f"Total parameters: {total_params:,}")

# Create a sentiment analysis pipeline for convenience
finbert_pipeline = pipeline(
    "sentiment-analysis",
    model=finbert_model,
    tokenizer=finbert_tokenizer,
    device=0 if device.type == 'cuda' else -1 if device.type == 'cpu' else 'mps'
)

print("\n✅ FinBERT loaded and ready for sentiment analysis!")

## 5. Tokenization for Financial Text

Understanding tokenization is crucial for financial NLP:
- Financial terms may be split into subwords
- Numbers and percentages have special handling
- Company names and tickers may be tokenized unexpectedly

In [ ]:
# Financial text examples for tokenization analysis
financial_texts = [
    "AAPL reported Q3 earnings of $1.26 EPS, beating estimates by 15%.",
    "The Fed raised rates by 75bps amid persistent inflation concerns.",
    "Goldman Sachs upgraded NVDA to 'Buy' with a $500 price target.",
    "Credit Suisse faces $4.7B loss from Archegos margin call.",
    "YoY revenue growth of 23% driven by strong SaaS adoption."
]

print("=" * 70)
print("FINANCIAL TEXT TOKENIZATION ANALYSIS")
print("=" * 70)

for text in financial_texts:
    tokens = finbert_tokenizer.tokenize(text)
    encoding = finbert_tokenizer(
        text,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=128,
        return_attention_mask=True,
        return_token_type_ids=True
    )
    
    print(f"\n📝 Text: {text}")
    print(f"   Tokens ({len(tokens)}): {tokens}")
    print(f"   Token IDs: {encoding['input_ids'][0].tolist()[:15]}...")
    print(f"   Attention Mask: {encoding['attention_mask'][0].tolist()[:15]}...")

In [ ]:
# Deep dive: Special tokens and padding
print("=" * 60)
print("SPECIAL TOKENS IN BERT/FinBERT")
print("=" * 60)

print(f"\n[CLS] token ID: {finbert_tokenizer.cls_token_id} -> '{finbert_tokenizer.cls_token}'")
print(f"[SEP] token ID: {finbert_tokenizer.sep_token_id} -> '{finbert_tokenizer.sep_token}'")
print(f"[PAD] token ID: {finbert_tokenizer.pad_token_id} -> '{finbert_tokenizer.pad_token}'")
print(f"[UNK] token ID: {finbert_tokenizer.unk_token_id} -> '{finbert_tokenizer.unk_token}'")
print(f"[MASK] token ID: {finbert_tokenizer.mask_token_id} -> '{finbert_tokenizer.mask_token}'")

# Demonstrate padding strategies
short_text = "Stock rises."
long_text = "Apple Inc. reported quarterly earnings that significantly exceeded Wall Street expectations."

print("\n" + "=" * 60)
print("PADDING STRATEGIES")
print("=" * 60)

# Pad to max length
encoded_padded = finbert_tokenizer(
    [short_text, long_text],
    padding='max_length',
    max_length=20,
    truncation=True,
    return_tensors='pt'
)

print(f"\nWith padding='max_length' (max_length=20):")
for i, text in enumerate([short_text, long_text]):
    tokens = finbert_tokenizer.convert_ids_to_tokens(encoded_padded['input_ids'][i])
    print(f"\n  Text: {text}")
    print(f"  Tokens: {tokens}")
    print(f"  Attention: {encoded_padded['attention_mask'][i].tolist()}")

print("\n💡 Key Points:")
print("  - [CLS] always at the beginning (used for classification)")
print("  - [SEP] marks end of sentence")
print("  - [PAD] tokens have attention_mask=0 (ignored by model)")

## 6. Financial Sentiment Classification

Now let's use FinBERT for real financial sentiment analysis tasks.

In [ ]:
# Sample financial news headlines for sentiment analysis
financial_headlines = [
    # Positive sentiment
    "Apple beats earnings expectations with record iPhone sales",
    "Tesla stock surges 15% after announcing breakthrough in battery technology",
    "Microsoft announces $60 billion share buyback program",
    "Amazon Web Services revenue grows 37% year-over-year",
    "JPMorgan reports strongest quarterly profit in company history",
    
    # Negative sentiment
    "Boeing faces new safety concerns after whistleblower revelations",
    "Credit Suisse stock plunges amid liquidity crisis fears",
    "Meta announces 11,000 layoffs as advertising revenue declines",
    "FTX files for bankruptcy after $8 billion shortfall discovered",
    "Evergrande misses bond payment deadline, default risk rises",
    
    # Neutral sentiment
    "Federal Reserve holds interest rates steady at current levels",
    "Quarterly earnings report scheduled for release next Tuesday",
    "Company announces leadership transition plan for 2024",
    "Stock market closes flat after mixed economic data",
    "Analysts maintain neutral rating on retail sector stocks"
]

def analyze_sentiment(texts, model, tokenizer, batch_size=8):
    """
    Analyze sentiment for a list of texts using FinBERT.
    
    Returns DataFrame with text, sentiment label, and confidence scores.
    """
    results = []
    model.eval()
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        
        # Tokenize batch
        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Get predictions
        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=-1)
            predictions = torch.argmax(probs, dim=-1)
        
        # Process results
        for j, text in enumerate(batch_texts):
            pred_label = model.config.id2label[predictions[j].item()]
            confidence = probs[j][predictions[j]].item()
            
            results.append({
                'text': text,
                'sentiment': pred_label,
                'confidence': confidence,
                'positive_prob': probs[j][2].item() if 2 in model.config.id2label else probs[j][0].item(),
                'negative_prob': probs[j][0].item() if 0 in model.config.id2label else probs[j][1].item(),
                'neutral_prob': probs[j][1].item() if 1 in model.config.id2label else probs[j][2].item()
            })
    
    return pd.DataFrame(results)

# Analyze sentiment
print("=" * 70)
print("FINANCIAL SENTIMENT ANALYSIS WITH FINBERT")
print("=" * 70)

sentiment_df = analyze_sentiment(financial_headlines, finbert_model, finbert_tokenizer)

# Display results
for idx, row in sentiment_df.iterrows():
    emoji = "🟢" if row['sentiment'] == 'positive' else "🔴" if row['sentiment'] == 'negative' else "⚪"
    print(f"\n{emoji} [{row['sentiment'].upper():^8}] (conf: {row['confidence']:.2%})")
    print(f"   {row['text'][:80]}...")

In [ ]:
# Visualize sentiment distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Sentiment counts
ax1 = axes[0]
sentiment_counts = sentiment_df['sentiment'].value_counts()
colors = {'positive': '#2ecc71', 'negative': '#e74c3c', 'neutral': '#95a5a6'}
bars = ax1.bar(sentiment_counts.index, sentiment_counts.values, 
               color=[colors[s] for s in sentiment_counts.index])
ax1.set_xlabel('Sentiment')
ax1.set_ylabel('Count')
ax1.set_title('Sentiment Distribution')
for bar, count in zip(bars, sentiment_counts.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             str(count), ha='center', va='bottom', fontweight='bold')

# Confidence distribution
ax2 = axes[1]
for sentiment in ['positive', 'negative', 'neutral']:
    subset = sentiment_df[sentiment_df['sentiment'] == sentiment]['confidence']
    if len(subset) > 0:
        ax2.hist(subset, bins=10, alpha=0.6, label=sentiment, color=colors[sentiment])
ax2.set_xlabel('Confidence Score')
ax2.set_ylabel('Frequency')
ax2.set_title('Confidence Distribution by Sentiment')
ax2.legend()

# Probability breakdown for sample texts
ax3 = axes[2]
sample_indices = [0, 5, 10]  # One positive, one negative, one neutral
x = np.arange(len(sample_indices))
width = 0.25

pos_probs = [sentiment_df.iloc[i]['positive_prob'] for i in sample_indices]
neg_probs = [sentiment_df.iloc[i]['negative_prob'] for i in sample_indices]
neu_probs = [sentiment_df.iloc[i]['neutral_prob'] for i in sample_indices]

ax3.bar(x - width, pos_probs, width, label='Positive', color=colors['positive'])
ax3.bar(x, neg_probs, width, label='Negative', color=colors['negative'])
ax3.bar(x + width, neu_probs, width, label='Neutral', color=colors['neutral'])
ax3.set_xticks(x)
ax3.set_xticklabels(['Headline 1', 'Headline 6', 'Headline 11'])
ax3.set_ylabel('Probability')
ax3.set_title('Probability Breakdown for Sample Headlines')
ax3.legend()
ax3.set_ylim(0, 1)

plt.tight_layout()
plt.show()

# Summary statistics
print("\n" + "=" * 60)
print("SENTIMENT ANALYSIS SUMMARY")
print("=" * 60)
print(f"\nTotal headlines analyzed: {len(sentiment_df)}")
print(f"\nSentiment breakdown:")
for sentiment, count in sentiment_counts.items():
    pct = count / len(sentiment_df) * 100
    print(f"  - {sentiment.capitalize()}: {count} ({pct:.1f}%)")
print(f"\nAverage confidence: {sentiment_df['confidence'].mean():.2%}")

## 7. Batch Processing Financial News

Efficient batch processing is essential for analyzing large volumes of financial text in production systems.

In [ ]:
class FinancialNewsDataset(Dataset):
    """
    PyTorch Dataset for financial news text.
    Handles tokenization and batching efficiently.
    """
    
    def __init__(self, texts, tokenizer, max_length=256, labels=None):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.labels = labels
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        
        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze()
        }
        
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
            
        return item

def batch_inference(texts, model, tokenizer, batch_size=16):
    """
    Efficient batch inference with progress bar.
    """
    dataset = FinancialNewsDataset(texts, tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    all_predictions = []
    all_probabilities = []
    
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Processing batches"):
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)
            
            probs = F.softmax(outputs.logits, dim=-1)
            preds = torch.argmax(probs, dim=-1)
            
            all_predictions.extend(preds.cpu().numpy())
            all_probabilities.extend(probs.cpu().numpy())
    
    return np.array(all_predictions), np.array(all_probabilities)

# Generate larger synthetic dataset for demonstration
np.random.seed(42)
synthetic_news = []
templates_positive = [
    "{company} reports record quarterly revenue of ${revenue}B",
    "{company} stock jumps {pct}% on strong earnings beat",
    "{company} raises full-year guidance amid robust demand",
    "Analysts upgrade {company} after impressive product launch",
    "{company} announces ${amount}B share buyback program"
]
templates_negative = [
    "{company} misses earnings estimates, stock falls {pct}%",
    "{company} faces regulatory probe over compliance issues",
    "{company} announces {num} layoffs amid restructuring",
    "{company} warns of supply chain disruptions affecting Q4",
    "Credit rating agency downgrades {company} to junk status"
]
templates_neutral = [
    "{company} schedules investor day for next month",
    "{company} appoints new CFO effective January 1",
    "{company} to release earnings report on Tuesday",
    "Trading volume in {company} remains average",
    "{company} maintains dividend at ${amount} per share"
]

companies = ['Apple', 'Microsoft', 'Google', 'Amazon', 'Tesla', 'Meta', 
             'Netflix', 'NVIDIA', 'JPMorgan', 'Goldman Sachs']

for _ in range(30):
    company = np.random.choice(companies)
    template = np.random.choice(templates_positive)
    synthetic_news.append(template.format(
        company=company, revenue=np.random.randint(20, 100),
        pct=np.random.randint(5, 20), amount=np.random.randint(10, 60), num=1000
    ))
    
for _ in range(30):
    company = np.random.choice(companies)
    template = np.random.choice(templates_negative)
    synthetic_news.append(template.format(
        company=company, revenue=np.random.randint(20, 100),
        pct=np.random.randint(5, 20), amount=np.random.randint(10, 60),
        num=np.random.randint(1000, 10000)
    ))
    
for _ in range(30):
    company = np.random.choice(companies)
    template = np.random.choice(templates_neutral)
    synthetic_news.append(template.format(
        company=company, revenue=np.random.randint(20, 100),
        pct=np.random.randint(5, 20), amount=f"{np.random.uniform(0.5, 3):.2f}", num=1000
    ))

print(f"Generated {len(synthetic_news)} synthetic news headlines")
print(f"\nSample headlines:")
for i in [0, 30, 60]:
    print(f"  - {synthetic_news[i]}")

In [ ]:
# Run batch inference
print("=" * 60)
print("BATCH INFERENCE ON SYNTHETIC NEWS")
print("=" * 60)

import time
start_time = time.time()

predictions, probabilities = batch_inference(
    synthetic_news, finbert_model, finbert_tokenizer, batch_size=16
)

elapsed_time = time.time() - start_time
print(f"\n✅ Processed {len(synthetic_news)} headlines in {elapsed_time:.2f} seconds")
print(f"   Throughput: {len(synthetic_news)/elapsed_time:.1f} headlines/second")

# Create results DataFrame
batch_results = pd.DataFrame({
    'text': synthetic_news,
    'prediction': [finbert_model.config.id2label[p] for p in predictions],
    'positive_prob': probabilities[:, 2] if 2 in finbert_model.config.id2label else probabilities[:, 0],
    'negative_prob': probabilities[:, 0] if 0 in finbert_model.config.id2label else probabilities[:, 1],
    'neutral_prob': probabilities[:, 1] if 1 in finbert_model.config.id2label else probabilities[:, 2],
    'confidence': np.max(probabilities, axis=1)
})

# Analyze results
print("\n" + "=" * 60)
print("BATCH RESULTS ANALYSIS")
print("=" * 60)

prediction_counts = batch_results['prediction'].value_counts()
print(f"\nPrediction distribution:")
for pred, count in prediction_counts.items():
    print(f"  {pred.capitalize()}: {count} ({count/len(batch_results)*100:.1f}%)")

# Show some predictions with their expected sentiment
print("\nSample predictions (first 30 were designed as positive):")
for i in [0, 1, 2]:
    row = batch_results.iloc[i]
    print(f"  📰 {row['text'][:60]}...")
    print(f"     Predicted: {row['prediction']} (conf: {row['confidence']:.2%})")

print("\nSample predictions (31-60 were designed as negative):")
for i in [30, 31, 32]:
    row = batch_results.iloc[i]
    print(f"  📰 {row['text'][:60]}...")
    print(f"     Predicted: {row['prediction']} (conf: {row['confidence']:.2%})")

## 8. Fine-tuning FinBERT on Custom Data

Fine-tuning adapts the pre-trained model to your specific financial domain or task. Key considerations:
- **Learning rate**: Lower than training from scratch (2e-5 to 5e-5)
- **Epochs**: Few epochs to avoid overfitting (2-4 typically)
- **Warmup**: Gradual learning rate increase prevents catastrophic forgetting

In [ ]:
# Create a labeled dataset for fine-tuning demonstration
# Labels: 0=negative, 1=neutral, 2=positive

labeled_data = []

# Positive examples (label=2)
positive_texts = [
    "Revenue exceeded analyst expectations by 15%",
    "Company announces special dividend of $2.50 per share",
    "Strong forward guidance lifted investor confidence",
    "Profit margins expanded to record levels",
    "Customer acquisition costs declined significantly",
    "Market share gains accelerated in key segments",
    "Cash flow generation surpassed all forecasts",
    "New product launch received exceptional reception",
    "Strategic partnership expected to boost revenue",
    "Operating efficiency improvements drove profit growth"
]
labeled_data.extend([(text, 2) for text in positive_texts])

# Negative examples (label=0)
negative_texts = [
    "Revenue missed consensus estimates by 8%",
    "Company suspends dividend amid cash concerns",
    "Weak guidance disappointed investors",
    "Profit margins contracted due to cost pressures",
    "Customer churn rate increased substantially",
    "Market share losses in core business",
    "Cash burn rate raised liquidity concerns",
    "Product recall will impact quarterly results",
    "Key partnership terminated unexpectedly",
    "Restructuring charges will pressure earnings"
]
labeled_data.extend([(text, 0) for text in negative_texts])

# Neutral examples (label=1)
neutral_texts = [
    "Quarterly results in line with expectations",
    "Management reiterated full-year guidance",
    "Board approved routine capital expenditure",
    "Annual shareholder meeting scheduled",
    "Company filed regular SEC disclosures",
    "Trading volume remained within normal range",
    "Analyst coverage initiated with hold rating",
    "Executive compensation disclosed in proxy",
    "Credit facility renewed at existing terms",
    "Industry conference presentation scheduled"
]
labeled_data.extend([(text, 1) for text in neutral_texts])

# Shuffle and split
np.random.shuffle(labeled_data)
texts = [item[0] for item in labeled_data]
labels = [item[1] for item in labeled_data]

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Training set: {len(train_texts)} samples")
print(f"Validation set: {len(val_texts)} samples")
print(f"\nLabel distribution (train):")
for label in [0, 1, 2]:
    count = train_labels.count(label)
    print(f"  Label {label}: {count}")

In [ ]:
class FineTuningDataset(Dataset):
    """Dataset class for fine-tuning with labels."""
    
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

def train_epoch(model, dataloader, optimizer, scheduler):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    
    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
    
    return total_loss / len(dataloader)

def evaluate(model, dataloader):
    """Evaluate model on validation set."""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)
            
            total_loss += outputs.loss.item()
            preds = torch.argmax(outputs.logits, dim=-1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch['labels'].numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return total_loss / len(dataloader), accuracy, f1

# Fine-tuning setup
print("=" * 60)
print("FINE-TUNING SETUP")
print("=" * 60)

# Load a fresh model for fine-tuning
finetune_model = AutoModelForSequenceClassification.from_pretrained(
    'ProsusAI/finbert',
    num_labels=3
)
finetune_model = finetune_model.to(device)

# Create datasets
train_dataset = FineTuningDataset(train_texts, train_labels, finbert_tokenizer)
val_dataset = FineTuningDataset(val_texts, val_labels, finbert_tokenizer)

# Hyperparameters
batch_size = 8
learning_rate = 2e-5
num_epochs = 3
warmup_ratio = 0.1

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Optimizer and scheduler
optimizer = AdamW(finetune_model.parameters(), lr=learning_rate, weight_decay=0.01)

total_steps = len(train_loader) * num_epochs
warmup_steps = int(total_steps * warmup_ratio)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

print(f"\nTraining configuration:")
print(f"  - Batch size: {batch_size}")
print(f"  - Learning rate: {learning_rate}")
print(f"  - Epochs: {num_epochs}")
print(f"  - Total steps: {total_steps}")
print(f"  - Warmup steps: {warmup_steps}")